In [1]:
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm

### Read the data

In [2]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [3]:
class Translator:
    def __init__(self):
        self.source_ranges_start = list()
        self.source_ranges_end = list()

        self.target_ranges_start = list()
        self.target_ranges_end = list()
    
    def add_ranges(self, start_target, start_source, range_length):
        self.source_ranges_start.append(start_source)
        self.source_ranges_end.append(start_source + range_length)

        self.target_ranges_start.append(start_target)
        self.target_ranges_end.append(start_target + range_length)

    def translate(self, source):
        for i in range(len(self.source_ranges_start)):
            if source >= self.source_ranges_start[i] and source < self.source_ranges_end[i]:
                return self.target_ranges_start[i] + (source - self.source_ranges_start[i])
        return source
    
    def reverse_translate(self, target):
        for i in range(len(self.target_ranges_start)):
            if target >= self.target_ranges_start[i] and target < self.target_ranges_end[i]:
                return self.source_ranges_start[i] + (target - self.target_ranges_start[i])

        # check if target in source range
        for i in range(len(self.source_ranges_start)):
            if target >= self.source_ranges_start[i] and target < self.source_ranges_end[i]:
                return False

        return target


In [4]:
working_file = file_content

initial_seeds = working_file[0].split(': ')[1].split(' ')
initial_seeds = [int(x) for x in initial_seeds]

all_mappings = list()

i = 2

while i < len(working_file):
    # if '-' in working_file[i]:
    if '-' in working_file[i]:

        translator = Translator()
        while (i+1) < len(working_file) and working_file[i+1] != '':
            mapping = working_file[i+1].split(' ')
            dest_range_start, src_range_start, range_length = [int(x) for x in mapping]
            translator.add_ranges(dest_range_start, src_range_start, range_length)
            i += 1
        all_mappings.append(translator)
    i += 1
lowest_location_number = np.inf
for seed in initial_seeds:
    current = seed
    for translator in all_mappings:
        current = translator.translate(current)

    if current < lowest_location_number:
        lowest_location_number = current
print(lowest_location_number)

51580674


### Part 2

In [5]:
D = open(fn).read().strip()
L = D.split('\n')

parts = D.split('\n\n')
seed, *others = parts
seed = [int(x) for x in seed.split(':')[1].split()]

class Function:
  def __init__(self, S):
    lines = S.split('\n')[1:] # throw away name
    # dst src sz
    self.tuples: list[tuple[int,int,int]] = [[int(x) for x in line.split()] for line in lines]
    #print(self.tuples)
  def apply_one(self, x: int) -> int:
    for (dst, src, sz) in self.tuples:
      if src<=x<src+sz:
        return x+dst-src
    return x

  # list of [start, end) ranges
  def apply_range(self, R):
    A = []
    for (dest, src, sz) in self.tuples:
      src_end = src+sz
      NR = []
      while R:
        # [st                                     ed)
        #          [src       src_end]
        # [BEFORE ][INTER            ][AFTER        )
        (st,ed) = R.pop()
        # (src,sz) might cut (st,ed)
        before = (st,min(ed,src))
        inter = (max(st, src), min(src_end, ed))
        after = (max(src_end, st), ed)
        if before[1]>before[0]:
          NR.append(before)
        if inter[1]>inter[0]:
          A.append((inter[0]-src+dest, inter[1]-src+dest))
        if after[1]>after[0]:
          NR.append(after)
      R = NR
    return A+R

Fs = [Function(s) for s in others]

P2 = []
pairs = list(zip(seed[::2], seed[1::2]))
for st, sz in pairs:
  # inclusive on the left, exclusive on the right
  # e.g. [1,3) = [1,2]
  # length of [a,b) = b-a
  # [a,b) + [b,c) = [a,c)
  R = [(st, st+sz)]
  for f in Fs:
    R = f.apply_range(R)
  #print(len(R))
  P2.append(min(R)[0])
print(min(P2))

99751240
